In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import pandas as pd
import os

In [3]:
deprel = "nummod"

In [4]:
root = ".../margendatud/"+deprel
fil = root + "/" + "project-9-at-2023-06-02-19-38-32b32179.json"
with open(fil, encoding="utf-8") as f:
    data = json.load(f)

In [5]:
fil2 = ".../train_dev_for_labelstudio/" + deprel + "/" + "ls_puupank_nummod_export_big_v1_fileinfo.txt"
with open(fil2, "r", encoding="utf-8") as f:
    fileinfo = f.readlines()
    

In [6]:
data[0]

{'id': 66292,
 'annotations': [{'id': 19288,
   'completed_by': 1,
   'result': [{'value': {'start': 71,
      'end': 76,
      'text': 'kahte',
      'labels': ['nummod_phrases']},
     'id': 'MuP111hZJ1',
     'from_name': 'label',
     'to_name': 'text',
     'type': 'labels',
     'origin': 'prediction'},
    {'value': {'choices': ['bound']},
     'id': '_kF5w_Turg',
     'from_name': 'review',
     'to_name': 'text',
     'type': 'choices',
     'origin': 'manual'}],
   'was_cancelled': False,
   'ground_truth': False,
   'created_at': '2023-04-27T06:41:16.998155Z',
   'updated_at': '2023-04-27T06:41:16.998187Z',
   'lead_time': 14.432,
   'prediction': {'id': 66291,
    'model_version': 'undefined',
    'created_ago': '0\xa0minutes',
    'result': [{'to_name': 'text',
      'from_name': 'label',
      'type': 'labels',
      'value': {'start': 71,
       'end': 76,
       'text': ['kahte'],
       'labels': ['nummod_phrases']}}],
    'score': None,
    'cluster': None,
    'neigh

In [7]:
all_data = []

for obj in data:
    idx = obj["id"]
    removed = obj["annotations"][0]["result"][0]["value"]["text"]
    if len(obj["annotations"][0]["result"]) == 1:
        print(idx)
    else:
        verdict = obj["annotations"][0]["result"][1]["value"]["choices"][0]
        sentence = obj["data"]["text"]
        
        remove_start = obj["annotations"][0]["result"][0]["value"]["start"]
        remove_end = obj["annotations"][0]["result"][0]["value"]["end"]
        short_sent = sentence[:remove_start] + sentence[remove_end:].strip()
        
        match = False
        for line in fileinfo[1:]:
            line = line.strip()
            if sentence.replace(" ", "") in line.replace(" ", "") and removed==line.split("\\")[2]:
                fname = line.split("\\")[0]
                cons_score = line.split("\\")[3]
                ual = line.split("\\")[4]
                la = line.split("\\")[5]
                root = ".../UDpuupank/UD2_11_udreposse/"
                if os.path.isfile(root+"Train/"+fname):
                    fpath = "Train/"+fname
                elif os.path.isfile(root+"Dev/"+fname):
                    fpath = "Dev/"+fname
                    
                match = True
                
                all_data.append([idx, fpath, sentence, remove_start, remove_end, removed, verdict, short_sent, cons_score, ual, la])    
                break
        
        #if not match:
        #    print(sentence)

        #all_data.append([idx, fpath, sentence, remove_start, remove_end, removed, verdict, short_sent, cons_score, ual, la])

66965
66968


In [8]:
df = pd.DataFrame(all_data, columns=["id", "fpath", "sentence", "remove_start", "remove_end", "removed", "type", "short_sent", "cons_score", "ual", "la"])


In [9]:
df

,id,fpath,sentence,remove_start,remove_end,removed,type,short_sent,cons_score,ual,la
0,66292,Train/aja_pm20001004_osa_2_ud211.conllu,Pühapäeva hilisõhtul enne keskööd märkas Lunin...,71,76,kahte,bound,Pühapäeva hilisõhtul enne keskööd märkas Lunin...,86.7,86.7,100.0
1,66293,Train/aja_pm19980926e_ud211.conllu,"Et kaks kanget korraga peeneks jahvatatakse , ...",3,7,kaks,bound,"Et kanget korraga peeneks jahvatatakse , ei ol...",100.0,100.0,100.0
2,66294,Train/tea_eesti_arst_2004_osa_2_ud211.conllu,Patsient on olnud kopsuarstide vaateväljas ena...,43,65,enam kui kahe ja poole,bound,Patsient on olnud kopsuarstide vaateväljas aas...,100.0,100.0,100.0
3,66295,Train/aja_luup200106_osa_2_ud211.conllu,Teine pool on egoistlik - lähema 10-20 aasta p...,26,38,lähema 10-20,bound,Teine pool on egoistlik - aasta pärast peaks h...,88.9,94.4,88.9
4,66296,Train/aja_horisont2000_osa_3_ud211.conllu,Kirjakeel aastast 1918,18,22,1918,bound,Kirjakeel aastast,100.0,100.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...
2735,69259,Train/aja_horisont2000_osa_3_ud211.conllu,Meil ei oleks võib-olla mõtet rääkida miljonit...,38,49,miljonitest,bound,Meil ei oleks võib-olla mõtet rääkida aastates...,55.6,55.6,83.3
2736,69260,Dev/ilu_ruben_ud211.conllu,See on sulle endale ka kasulik : kui tööleping...,204,209,kolme,bound,See on sulle endale ka kasulik : kui tööleping...,85.0,87.5,95.0
2737,69261,Train/tea_AA_05_6_ud211.conllu,"Arvestades aga , et isegi artiklis [ 1 ] suude...",35,40,[ 1 ],bound,"Arvestades aga , et isegi artiklis suudeti pea...",89.1,95.7,93.5
2738,69262,Train/tea_AA_05_6_ud211.conllu,"Arvestades aga , et isegi artiklis [ 1 ] suude...",133,139,[ 10 ],bound,"Arvestades aga , et isegi artiklis [ 1 ] suude...",89.1,95.7,93.5


In [10]:
df_free = df.loc[df['type'] == "free"]
df_free

,id,fpath,sentence,remove_start,remove_end,removed,type,short_sent,cons_score,ual,la
6,66298,Train/aja_ee199920_osa_1_ud211.conllu,"Leiti , et Eesti Ekspress ei kaota midagi , ku...",57,60,ühe,free,"Leiti , et Eesti Ekspress ei kaota midagi , ku...",100.0,100.0,100.0
28,66320,Train/aja_pm20001004_osa_1_ud211.conllu,Kalevi hinnangul on kooli kaheks jagamine kasu...,26,32,kaheks,free,Kalevi hinnangul on kooli jagamine kasulik vai...,95.5,95.5,100.0
31,66323,Train/aja_ee200110_osa_9_ud211.conllu,Nüüd töötas Liis ühe päeva ühes Tallinna keemi...,17,20,ühe,free,Nüüd töötas Liis päeva ühes Tallinna keemilise...,100.0,100.0,100.0
41,66333,Train/tea_AA_05_6_ud211.conllu,Selle mõneti tüütu protsessi peab üks kord läb...,34,37,üks,free,Selle mõneti tüütu protsessi peab kord läbi te...,87.5,100.0,87.5
57,66349,Train/aja_luup200202_osa_2_ud211.conllu,Maxima on maksimaalne ! rõkkasid hollandlased ...,96,104,02.02.02,free,Maxima on maksimaalne ! rõkkasid hollandlased ...,63.6,63.6,90.9
...,...,...,...,...,...,...,...,...,...,...,...
2683,69145,Dev/aja_ee199920_osa4_ud211.conllu,Valida on DOCG Chianti Castellani 1997 noore j...,34,38,1997,free,Valida on DOCG Chianti Castellani noore ja mah...,53.6,53.6,75.0
2684,69147,Train/tea_AA_05_6_ud211.conllu,2. Aidatakse kaasa alamprojektide 1 - 3 loodud...,34,39,1 - 3,free,2. Aidatakse kaasa alamprojektide loodud vahen...,64.3,64.3,92.9
2701,69197,Train/tea_AA_05_6_ud211.conllu,Seejuures väljub õpetamisprotsess ühe haridusa...,34,37,ühe,free,Seejuures väljub õpetamisprotsess haridusasutu...,95.7,95.7,100.0
2713,69209,Train/aja_horisont2000_osa_1_ud211.conllu,Juulis 1891 saabus Kroonlinna Prantsuse sõjala...,7,11,1891,free,Juulis saabus Kroonlinna Prantsuse sõjalaevast...,85.7,85.7,85.7


In [11]:
df_free.to_csv(f"deprel_free_analysis/csv_files/{deprel}_free.csv", sep=";", index=False)

In [12]:
df_free_cons = df_free[df_free["cons_score"]=="100.0"]
df_free_cons

,id,fpath,sentence,remove_start,remove_end,removed,type,short_sent,cons_score,ual,la
6,66298,Train/aja_ee199920_osa_1_ud211.conllu,"Leiti , et Eesti Ekspress ei kaota midagi , ku...",57,60,ühe,free,"Leiti , et Eesti Ekspress ei kaota midagi , ku...",100.0,100.0,100.0
31,66323,Train/aja_ee200110_osa_9_ud211.conllu,Nüüd töötas Liis ühe päeva ühes Tallinna keemi...,17,20,ühe,free,Nüüd töötas Liis päeva ühes Tallinna keemilise...,100.0,100.0,100.0
102,66394,Dev/arborest_dev_ud211.conllu,Pealkiri käib üle kolme veeru .,18,23,kolme,free,Pealkiri käib üle veeru .,100.0,100.0,100.0
110,66402,Train/ilu_vilep_ud211.conllu,"Sõitnud veel ühe täisringi linnale peale , läk...",8,16,veel ühe,free,"Sõitnud täisringi linnale peale , läks ta oma ...",100.0,100.0,100.0
116,66408,Train/aja_ee200110_osa_11_ud211.conllu,Ja seda siis ühe loo eest .,13,16,ühe,free,Ja seda siis loo eest .,100.0,100.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...
2535,68876,Train/tea_dr8047.conllu,Tabel 3 .,6,7,3,free,Tabel .,100.0,100.0,100.0
2611,69015,Train/arborest_ud211.conllu,Oktoobris 1990 pidi Eesti esindus sõitma USA p...,10,14,1990,free,Oktoobris pidi Eesti esindus sõitma USA presid...,100.0,100.0,100.0
2614,69018,Dev/aja_luup200009_ud211.conllu,"Kui räägime sotsiaalsest tellimusest , siis on...",53,66,neist kolmest,free,"Kui räägime sotsiaalsest tellimusest , siis on...",100.0,100.0,100.0
2620,69024,Train/aja_ee199920_osa_1_ud211.conllu,Pere tahtis kaasata täiendavaid laene paarisaj...,38,60,paarisaja miljoni eest,free,"Pere tahtis kaasata täiendavaid laene , et ren...",100.0,100.0,100.0


In [13]:
df_free_cons.to_csv(f"deprel_free_analysis/csv_files/{deprel}_free_conserved.csv", sep=";", index=False)

In [10]:
df_bound = df.loc[df['type'] == "bound"]
df_bound

,id,fpath,sentence,remove_start,remove_end,removed,type,short_sent,cons_score,ual,la
0,66292,Train/aja_pm20001004_osa_2_ud211.conllu,Pühapäeva hilisõhtul enne keskööd märkas Lunin...,71,76,kahte,bound,Pühapäeva hilisõhtul enne keskööd märkas Lunin...,86.7,86.7,100.0
1,66293,Train/aja_pm19980926e_ud211.conllu,"Et kaks kanget korraga peeneks jahvatatakse , ...",3,7,kaks,bound,"Et kanget korraga peeneks jahvatatakse , ei ol...",100.0,100.0,100.0
2,66294,Train/tea_eesti_arst_2004_osa_2_ud211.conllu,Patsient on olnud kopsuarstide vaateväljas ena...,43,65,enam kui kahe ja poole,bound,Patsient on olnud kopsuarstide vaateväljas aas...,100.0,100.0,100.0
3,66295,Train/aja_luup200106_osa_2_ud211.conllu,Teine pool on egoistlik - lähema 10-20 aasta p...,26,38,lähema 10-20,bound,Teine pool on egoistlik - aasta pärast peaks h...,88.9,94.4,88.9
4,66296,Train/aja_horisont2000_osa_3_ud211.conllu,Kirjakeel aastast 1918,18,22,1918,bound,Kirjakeel aastast,100.0,100.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...
2735,69259,Train/aja_horisont2000_osa_3_ud211.conllu,Meil ei oleks võib-olla mõtet rääkida miljonit...,38,49,miljonitest,bound,Meil ei oleks võib-olla mõtet rääkida aastates...,55.6,55.6,83.3
2736,69260,Dev/ilu_ruben_ud211.conllu,See on sulle endale ka kasulik : kui tööleping...,204,209,kolme,bound,See on sulle endale ka kasulik : kui tööleping...,85.0,87.5,95.0
2737,69261,Train/tea_AA_05_6_ud211.conllu,"Arvestades aga , et isegi artiklis [ 1 ] suude...",35,40,[ 1 ],bound,"Arvestades aga , et isegi artiklis suudeti pea...",89.1,95.7,93.5
2738,69262,Train/tea_AA_05_6_ud211.conllu,"Arvestades aga , et isegi artiklis [ 1 ] suude...",133,139,[ 10 ],bound,"Arvestades aga , et isegi artiklis [ 1 ] suude...",89.1,95.7,93.5


In [11]:
df_bound.to_csv(f"deprel_free_analysis/csv_files/{deprel}_bound.csv", sep=";", index=False)

In [12]:
df_comma = df.loc[df['type'] == "redundant comma"]
df_comma

,id,fpath,sentence,remove_start,remove_end,removed,type,short_sent,cons_score,ual,la
45,66337,Train/tea_toohoive.conllu,Nõnda väheneks tööpuuduse määr vastuseks produ...,75,79,2001,redundant comma,Nõnda väheneks tööpuuduse määr vastuseks produ...,83.3,91.7,91.7
98,66390,Train/tea_toohoive.conllu,Samuti on toimunud Euroopas oluline tööhõive v...,92,96,1999,redundant comma,Samuti on toimunud Euroopas oluline tööhõive v...,92.3,92.3,100.0
658,66952,Train/aja_horisont2000_osa_3_ud211.conllu,"Kui siirdame selleks vaid ühe geeni , siis pea...",86,100,vaid seda ühte,redundant comma,"Kui siirdame selleks vaid ühe geeni , siis pea...",100.0,100.0,100.0
1480,67781,Train/tea_toohoive.conllu,Kuna töökohtade loomine ja hävitamine sõltub t...,244,248,2001,redundant comma,Kuna töökohtade loomine ja hävitamine sõltub t...,100.0,100.0,100.0
1515,67816,Train/tea_toohoive.conllu,"SKP ja tööhõive trendid näitavad seda , et hoi...",199,203,1999,redundant comma,"SKP ja tööhõive trendid näitavad seda , et hoi...",78.4,81.1,86.5
1644,67945,Train/tea_toohoive.conllu,"( Trehan , 2001 ) .",11,15,2001,redundant comma,"( Trehan , ) .",100.0,100.0,100.0
1916,68224,Train/tea_toohoive.conllu,Lisaks on kõrgtehnoloogia kasutamisel eraettev...,122,126,1995,redundant comma,Lisaks on kõrgtehnoloogia kasutamisel eraettev...,94.1,100.0,94.1
2073,68382,Train/tea_toohoive.conllu,Samuti võib töökohtade vähese loomise põhjusek...,120,124,1999,redundant comma,Samuti võib töökohtade vähese loomise põhjusek...,100.0,100.0,100.0
2074,68383,Train/aja_horisont2000_osa_2_ud211.conllu,Sikhide kuulsus ja mõju maailmale on kaugelt s...,74,87,", 20 miljonit",redundant comma,Sikhide kuulsus ja mõju maailmale on kaugelt s...,100.0,100.0,100.0
2551,68896,Train/tea_toohoive.conllu,Madalad kasvumäärad võivad viia aga väiksema p...,172,176,1999,redundant comma,Madalad kasvumäärad võivad viia aga väiksema p...,100.0,100.0,100.0


In [13]:
df_comma.to_csv(f"deprel_free_analysis/csv_files/{deprel}_redundant_comma.csv", sep=";", index=False)

In [14]:
df_un = df.loc[df['type'] == "unnatural"]
df_un

,id,fpath,sentence,remove_start,remove_end,removed,type,short_sent,cons_score,ual,la
225,66517,Train/aja_luup200106_osa_4_ud211.conllu,Aja jooksul tabab politsei kümneid vargaid .,27,34,kümneid,unnatural,Aja jooksul tabab politsei vargaid .,83.3,83.3,83.3
390,66682,Dev/tea_eesti_arst_2004_osa_1_ud211.conllu,Uuritavad saadeti tsütogeneetilisele uuringule...,176,177,1,unnatural,Uuritavad saadeti tsütogeneetilisele uuringule...,None,None,None
758,67055,Dev/aja_epl20070812_ud211.conllu,Poiste arvestuses võitis esikoha Thorbjoem Sch...,168,170,30,unnatural,Poiste arvestuses võitis esikoha Thorbjoem Sch...,88.0,92.0,88.0
1024,67321,Train/aja_ml200247_osa_2_ud211.conllu,"Üks turumajanduse reegleid on küll see , et to...",138,144,ühtegi,unnatural,"Üks turumajanduse reegleid on küll see , et to...",92.9,92.9,96.4
2688,69184,Train/aja_ee199920_osa_2_ud211.conllu,Ma ei müünud sel ajal ühtki aktsiat .,22,27,ühtki,unnatural,Ma ei müünud sel ajal aktsiat .,100.0,100.0,100.0


In [15]:
df_un.to_csv(f"deprel_free_analysis/csv_files/{deprel}_unnatural.csv", sep=";", index=False)

In [16]:
df_dub = df.loc[df['type'] == "dubious"]
df_dub

,id,fpath,sentence,remove_start,remove_end,removed,type,short_sent,cons_score,ual,la


In [17]:
df_dub.to_csv(f"deprel_free_analysis/csv_files/{deprel}_dubious.csv", sep=";", index=False)

In [18]:
df_red = df.loc[df['type'] == "other redundant punctuation"]
df_red

,id,fpath,sentence,remove_start,remove_end,removed,type,short_sent,cons_score,ual,la
1023,67320,Train/aja_luup200202_osa_1_ud211.conllu,Sama moodi kui keskmine aktiivravi voodipäeva ...,55,58,"5,1",other redundant punctuation,Sama moodi kui keskmine aktiivravi voodipäeva ...,55.6,55.6,66.7
2581,68957,Train/tea_toohoive.conllu,"( Helemäe , Saar , Vöörmann , 1998 : 25 ) .",37,39,25,other redundant punctuation,"( Helemäe , Saar , Vöörmann , 1998 : ) .",100.0,100.0,100.0


In [19]:
df_red.to_csv(f"deprel_free_analysis/csv_files/{deprel}_redundant_punct.csv", sep=";", index=False)

In [ ]:
df_bound = df.loc[df['type'] == "bound"]
df_bound

df_bound.to_csv(f"deprel_free_analysis/csv_files/{deprel}_bound.csv", sep=";", index=False)

df_comma = df.loc[df['type'] == "redundant comma"]
df_comma

df_comma.to_csv(f"deprel_free_analysis/csv_files/{deprel}_redundant_comma.csv", sep=";", index=False)

df_un = df.loc[df['type'] == "unnatural"]
df_un

df_un.to_csv(f"deprel_free_analysis/csv_files/{deprel}_unnatural.csv", sep=";", index=False)

df_dub = df.loc[df['type'] == "dubious"]
df_dub

df_dub.to_csv(f"deprel_free_analysis/csv_files/{deprel}_dubious.csv", sep=";", index=False)

df_red = df.loc[df['type'] == "other redundant punctuation"]
df_red

df_red.to_csv(f"deprel_free_analysis/csv_files/{deprel}_redundant_punct.csv", sep=";", index=False)

## Vabade laienditega laused conll formaadis failidesse

In [11]:
from estnltk.converters.conll.conll_exporter import sentence_to_conll
from estnltk import Text
from estnltk.taggers import WhiteSpaceTokensTagger
from estnltk.taggers import PretokenizedTextCompoundTokensTagger
from estnltk_neural.taggers.syntax.stanza_tagger.stanza_tagger import StanzaSyntaxTagger

2023-06-14 08:29:21.853585: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-14 08:29:22.256996: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-14 08:29:23.727738: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-06-14 08:29:23.728160: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

In [12]:
model_path=".../lib/python3.9/site-packages/estnltk_neural/taggers/syntax/stanza_tagger/stanza_resources/"
input_type="morph_extended"
stanza_tagger = StanzaSyntaxTagger(input_type=input_type, input_morph_layer=input_type, add_parent_and_children=True, resources_path=model_path)

In [13]:
tokens_tagger = WhiteSpaceTokensTagger()
compound_tokens_tagger = PretokenizedTextCompoundTokensTagger()

In [14]:
from tqdm import tqdm

In [15]:
shortened_sents_data_dev = []
original_sents_data_dev = []
shortened_sents_data_train = []
original_sents_data_train = []

for row in tqdm(df_free.iterrows()):

    file = row[1][1]
    
    folder = file.split("/")[0]
    #print(folder)
    #break
    
    with open(f".../UDpuupank/UD2_11_udreposse/{file}") as f:
        conll = f.readlines()
        
    removed = row[1][5].split(" ")
    remove_start = row[1][3]
    remove_end = row[1][4]
    original_sent = row[1][2]
    sent = row[1][2].replace(" ", "")

    important_data = []

    # collect rows related to this sentence
    start = False
    for i, line in enumerate(conll):
        line = line.strip()
        mod_line = line.replace(" ", "")
        if start and not line.startswith("#"):
            important_data.append(line)
        elif start and (line=="" or line.startswith("#")):
            break    
        elif line.startswith("#") and sent in mod_line:
            start = True
            important_data.append(conll[i-1].strip())
            important_data.append(line)
            
            
    
                
            
    
    # find lines with removed words
    if len(important_data) != 0:
        
        #if "sent_id" in important_data[0]:
        #    sent_id = important_data[0].split("=")[1].strip()
        #    if sent_id in sent_dict.keys():
        if folder == "Train":
            original_sents_data_train.append(important_data)    
        elif folder == "Dev":
            original_sents_data_dev.append(important_data)
        else:
            print(f"folder {file} problem")
            break
        
        
        short_data_train = []
        short_data_dev = []
        
        short_data_pre = []
        cut_word_ids = []
        short_data_pre.append(important_data[0].strip())

        remove_count = 0
        for line in important_data[2:]:
            #print(line)

            checked = False  
            if line != "" and "sent_id =" not in line and "text =" not in line:
                added = len(line.split("\t")[1].strip() + " ")
                
                if line.split("\t")[1] in removed:
                    if len(cut_word_ids)==0 and remove_count == remove_start:
                        cut_word_ids.append(line.split("\t")[0])
                    elif len(cut_word_ids)!=0 and remove_count <= remove_end:
                        cut_word_ids.append(line.split("\t")[0])
                    checked = True
                remove_count += added
            #if not checked:    
            #    short_data.append(line+"\n")

        short_sentence = original_sent[:remove_start] + original_sent[remove_end+1:]

        txt = Text(short_sentence)
        
        tokens_tagger.tag(txt)
        compound_tokens_tagger.tag(txt)
        txt.tag_layer("morph_extended")
        stanza_tagger.tag(txt)
        res = sentence_to_conll(txt.sentences[0], txt["stanza_syntax"], udpipe=True)
        res2 = res.split("\n")
        
        
        short_data_pre.append("# cut_word_ids = " + ",".join(cut_word_ids))
        short_data_pre.append('# ud_version = 2.11')
        short_sent_text = "# text = " + " ".join([span.text for span in txt.words])
        short_data_pre.append(short_sent_text)
        
        
        #if "sent_id" in important_data[0]:
        #    sent_id = important_data[0].split("=")[1].strip()
        #    if sent_id in sent_dict.keys():
        if folder == "Train":
            short_data_train = short_data_pre + res2
            shortened_sents_data_train += [short_data_train]    
        elif folder == "Dev":
            short_data_dev = short_data_pre + res2
            shortened_sents_data_dev += [short_data_dev]
        
    #print(original_sent)
    #print(important_data)
    #print("\n",short_sent_text)
    #print(sentence_to_conll(row[1][7][:-2] + row[1][7][-2:].strip()))
    
    #print(short_data)
    
    #break
    
    
    #else:
    #    print("PROBLEEMIGA:", row[1][2])
    
#shortened_sents_data




364it [00:55,  6.55it/s]


In [16]:
sents_to_file = 2000

left = len(shortened_sents_data_dev) % sents_to_file
files = int(len(shortened_sents_data_dev)/sents_to_file)
if left != 0:
    files += 1

for i in range(files):
    with open(f"deprel_free_analysis/Dev/{deprel}_cut.conllu", "w", encoding="utf-8") as f:
        for obj in shortened_sents_data_dev[i*sents_to_file:(i+1)*sents_to_file]:
            for line in obj:
                f.write(line)
                f.write("\n")
                
    with open(f"deprel_free_analysis/Dev/{deprel}_original.conllu", "w", encoding="utf-8") as f:
        for obj in original_sents_data_dev[i*sents_to_file:(i+1)*sents_to_file]:
            for line in obj:
                f.write(line)
                f.write("\n")
                
                
left = len(shortened_sents_data_train) % sents_to_file
files = int(len(shortened_sents_data_train)/sents_to_file)
if left != 0:
    files += 1                
for i in range(files):
    with open(f"deprel_free_analysis/Train/{deprel}_cut.conllu", "w", encoding="utf-8") as f:
        for obj in shortened_sents_data_train[i*sents_to_file:(i+1)*sents_to_file]:
            for line in obj:
                f.write(line)
                f.write("\n")
                
    with open(f"deprel_free_analysis/Train/{deprel}_original.conllu", "w", encoding="utf-8") as f:
        for obj in original_sents_data_train[i*sents_to_file:(i+1)*sents_to_file]:
            for line in obj:
                f.write(line)
                f.write("\n")